## 1. Why RNN?
**RNN** architecture reflects the order of words in learning by **dealing with input text sequentially**

**RNN**은 **입력값을 순차적으로 처리함**으로써 단어의 등장 순서를 학습에 반영하는 아키텍쳐

<img width="604" alt="image" src="https://user-images.githubusercontent.com/35142536/60572653-06c72f00-9db1-11e9-999d-e4e70f7e4ec9.png">

Most importantly, green boxes are **hidden state**, the **memory** of RNN.

네트워크의 기억은 지금까지의 입력 데이터를 요약한 정보라고 볼 수 있습니다. 새로운 입력이 들어올때마다 네트워크는 자신의 기억을 조금씩 수정합니다. 결국 입력을 모두 처리하고 난 후 네트워크에게 남겨진 기억은 시퀀스 전체를 요약하는 정보가 됩니다.

## 2. Practice Code

In [1]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

In [7]:
sentences = ["i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w : i for i, w in enumerate(word_list)}
print(word_dict)
number_dict = {i : w for i,w in enumerate(word_list)}
print(number_dict)
n_class = len(word_dict)

['i', 'like', 'dog', 'i', 'love', 'coffee', 'i', 'hate']
{'dog': 0, 'milk': 1, 'like': 2, 'hate': 3, 'love': 4, 'coffee': 5, 'i': 6}
{0: 'dog', 1: 'milk', 2: 'like', 3: 'hate', 4: 'love', 5: 'coffee', 6: 'i'}


In [3]:
# TextRNN parameter
n_step = 2 # n of cells
# hidden state is memory
n_hidden = 5 # n of hidden units in one cell

def make_batch(sentences):
    input_batch = []
    target_batch = []
    
    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]
        
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(np.eye(n_class)[target])
        
    return input_batch, target_batch

In [4]:
# model
X = tf.placeholder(tf.float32, [None, n_step, n_class]) # [batch_size, n_step, n_class]
Y = tf.placeholder(tf.float32, [None, n_class]) # [batch_size, n_class]

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

# outputs : [batch_size, n_step, n_hidden]
outputs = tf.transpose(outputs, [1,0,2]) # [n_step, batch_size, n_hidden]
outputs = outputs[-1] # [batch_size, n_hidden]
model = tf.matmul(outputs, W) + b # model : [batch_size, n_class]

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

prediction = tf.cast(tf.argmax(model,1), tf.int32)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


In [5]:
# training
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

input_batch, target_batch = make_batch(sentences)

for epoch in range(5000):
    _, loss = sess.run([optimizer, cost], feed_dict={X : input_batch, Y : target_batch})
    if (epoch + 1)%500 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

input = [sen.split()[:2] for sen in sentences]

predict = sess.run([prediction], feed_dict={X: input_batch})
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n] for n in predict[0]])

Epoch: 0500 cost = 0.187387
Epoch: 1000 cost = 0.050126
Epoch: 1500 cost = 0.021309
Epoch: 2000 cost = 0.011298
Epoch: 2500 cost = 0.006754
Epoch: 3000 cost = 0.004342
Epoch: 3500 cost = 0.002927
Epoch: 4000 cost = 0.002037
Epoch: 4500 cost = 0.001450
Epoch: 5000 cost = 0.001050
[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
